## Phase 3 — Exploratory Data Analysis (EDA)

Purpose: turn the cleaned & engineered dataset into concise, business-grade insights. Produce repeatable outputs (tables, figures, a short narrative) that answer the stakeholder questions we agreed on (stocking, discontinuation, quality, discount effectiveness).

OVERALL GAMEPLAN (order of execution)

1. Brand Performance Analysis (ranking + validation)

2. Category Insights (aggregations by category_group)

3. Price-Tier & Discount Analysis (price_tier × discount_tier × sentiment)

4. Discount Effectiveness (do deeper discounts improve sentiment/ratings?)

5. Ultra-Discount Deep Dive (inspect ultra_discount cases individually)

6. Sentiment vs Rating Alignment (text vs numeric consistency check)

7. Final packaging: visuals, tables, and executive summary

In [1]:
import pandas as pd
import numpy as np
import re
from matplotlib import pyplot  

In [2]:
df_3 = pd.read_csv("/workspaces/Amazon-Sales-data-analysis/notebooks/cleaned_data_2.csv") 
df_3.head()

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,review_id,...,discount_percentage_num,discount_ratio,price_difference,ultra_discount,brands,price_tier,discount_tier,pos_word,neg_word,customer_sentiment
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories,399.0,1099.0,64%,4.2,24269.0,High Compatibility : Compatible With iPhone 12...,"R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...",...,64,0.363057,700.0,False,Wayona,Mid,High,True,False,satisfied
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories,199.0,349.0,43%,4.0,43994.0,"Compatible with all Type C enabled devices, be...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...",...,43,0.570201,150.0,False,Ambrane,Low,Mid,True,True,Mixed
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories,199.0,1899.0,90%,3.9,7928.0,【 Fast Charger& Data Sync】-With built-in safet...,"R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...",...,90,0.104792,1700.0,False,Sounce,Mid,High,True,False,satisfied
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories,329.0,699.0,53%,4.2,94363.0,The boAt Deuce USB 300 2 in 1 cable is compati...,"R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...",...,53,0.470672,370.0,False,boAt,Low,Mid,True,True,Mixed
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories,154.0,399.0,61%,4.2,16905.0,[CHARGE & SYNC FUNCTION]- This cable comes wit...,"R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...",...,61,0.385965,245.0,False,Portronics,Low,High,True,True,Mixed


In [ ]:
# Most products can be derived through the product_id, but this isnt a worthwhile metric.
# Its little and not normalized across the dataset, therefore, i dont plan on using it!

df_3["product_id"].value_counts()

product_id
B08HDJ86NZ    3
B07JW9H4J1    3
B07XLCFSSN    3
B08CF3D7QR    3
B083342NKJ    3
             ..
B07WFPMGQQ    1
B09LHYZ3GJ    1
B0BD3T6Z1D    1
B07WHSJXLF    1
B09QS9X16F    1
Name: count, Length: 1351, dtype: int64

### 1. BRAND PERFORMANCE   

**Goal: surface brands to promote, partner with, or monitor.**

What to compute (brand-level):
 
- n_reviews_brand = total reviews across brand (sum of rating_count or count of review rows).

- avg_rating_brand = mean rating across all brand reviews.

- percent_satisfied_brand = total satisfied reviews / total reviews for brand.

- avg_discount_ratio_brand and avg_price_diff_brand (pricing behavior).


**How to rank / filters:**

Ignore brands with very low sample (set a threshold, e.g., n_reviews_brand >= 10).

**Create lists:**

- Top brands: high avg_rating_brand, high percent_satisfied_brand, healthy n_reviews_brand.

- At-risk brands: high discounts + low satisfaction or low avg_rating.

**Validation check**

- Confirm brand extraction correctness (manual sample of 20 names to catch false positives like "The", "New", etc.).

**Suggested visuals & deliverables:**

- Bar: Top 10 brands by avg_rating_brand (filename: reports/brands_top10_rating.png)

- Scatter: avg_rating_brand vs n_reviews_brand (bubble=size=revenue_proxy) (reports/brands_rating_vs_volume.png)

- CSV: reports/brands_summary.csv

Narration excerpt:
“Brand X leads in satisfaction with Y% satisfied and strong volume; Brand Z requires attention — high discounting but low satisfaction suggesting quality or listing issues.

In [4]:
# total number of times each brands was mentioned 
df_3["brands"].value_counts() 

brands
boAt            67
Samsung         36
AmazonBasics    33
Portronics      31
Ambrane         29
                ..
KONVIO           1
Raffles          1
KNYUC            1
INKULTURE        1
Macmillan        1
Name: count, Length: 437, dtype: int64

In [5]:
# Grouping of each respective brands in respect to its total amount of reviews 
brands_rev = df_3.groupby("brands", as_index=False).sum().sort_values("rating_count", ascending=False) 
n_reviews_brand = brands_rev[["brands", "rating_count"]].reset_index(drop=True)

#List of top 10 brands with the highest review across the dataframe  
n_reviews_brand.head()   

,brands,rating_count
0,boAt,4416568.0
1,Redmi,1878482.0
2,AmazonBasics,1808818.0
3,SanDisk,1426074.0
4,Amazon,1213138.0


In [6]:
# Top 10 brands with the lowest reviews 
n_reviews_brand.tail()

,brands,rating_count
432,Syncwire,5.0
433,Longway,4.0
434,VAPJA®,4.0
435,Khaitan,2.0
436,NGI,2.0


In [7]:
# Averages of the total ratings present within each brands, ranked in respect to its rating_count    
brands_mean = df_3.groupby("brands").agg(mean_rating = ("rating" , "mean"),total_rating_count = ("rating_count", "sum"))
avg_rating_brand = brands_mean.sort_values("total_rating_count", ascending=False).reset_index()  

# Most reviewed and well rated brands   
avg_rating_brand.head() 

,brands,mean_rating,total_rating_count
0,boAt,4.080597,4416568.0
1,Redmi,4.053846,1878482.0
2,AmazonBasics,4.306061,1808818.0
3,SanDisk,4.370588,1426074.0
4,Amazon,4.140000,1213138.0


In [8]:
# least reviewd and lowest performing brands 
avg_rating_brand.tail()

,brands,mean_rating,total_rating_count
432,Syncwire,5.0,5.0
433,Longway,3.7,4.0
434,VAPJA®,3.6,4.0
435,Khaitan,2.0,2.0
436,NGI,3.1,2.0


In [9]:
# Mapping sentiments to numerical representations
mapping = {"satisfied": 1.0, "Mixed": 0, "dissatisfied": -1.0}
df_3["sentiment_value"] = df_3["customer_sentiment"].map(mapping) 

In [ ]:
brand_sentiment = (df_3.groupby("brands")
                  .agg(Total_sentiment_score=("sentiment_value", "count"),total_satisfied=("sentiment_value", lambda x: (x == 1).sum()),
                       total_mixed=("sentiment_value", lambda x: (x == 0).sum()))
                  .query("Total_sentiment_score >= 10")      
)           
# Storing the percentage of satisfied customer in a variable        
satisfied_percent = (brand_sentiment["total_satisfied"] / brand_sentiment["Total_sentiment_score"] * 100)
                                                                     
# Final Satisfaction Scoring metric for brands    
brand_sentiment["satisfaction_index"] = satisfied_percent * np.log1p(brand_sentiment["Total_sentiment_score"])

# Top Brands with the most satisfied customer
Top_brands_satisfied = brand_sentiment.sort_values("satisfaction_index", ascending=False).reset_index()

Top_brands_satisfied

,brands,Total_sentiment_score,total_satisfied,total_mixed,satisfaction_index
0,AmazonBasics,33,25,8,267.148525
1,pTron,13,13,0,263.905733
2,Duracell,20,15,5,228.339183
3,Noise,24,17,7,228.003704
4,Wayona,24,17,7,228.003704
5,Portronics,31,20,11,223.595865
6,STRIFF,11,9,1,203.310544
7,Classmate,11,9,2,203.310544
8,Bajaj,24,15,9,201.179739
9,AGARO,14,10,4,193.432157


In [ ]:
brand_sentiment = (df_3.groupby("brands")
                  .agg(Total_sentiment_score=("sentiment_value", "count"),total_dissatisfied=("sentiment_value", lambda x: (x == -1).sum()),
                       total_mixed=("sentiment_value", lambda x: (x == 0).sum()))
                  .query("Total_sentiment_score >= 10")      
)           
# Storing the percentage of dissatisfied customer in a variable        
dissatisfied_percent = (brand_sentiment["total_dissatisfied"] / brand_sentiment["Total_sentiment_score"] * 100)
                                                                     
# Final scoring metric of dissatisfied customer in respect to brands     
brand_sentiment["dissatisfaction_index"] = dissatisfied_percent * np.log1p(brand_sentiment["Total_sentiment_score"])

# Top Brands with the most dissatisfied customer
Top_brands_dissatisfied = brand_sentiment.sort_values("dissatisfaction_index", ascending=False).reset_index()

Top_brands_dissatisfied

,brands,Total_sentiment_score,total_dissatisfied,total_mixed,dissatisfaction_index
0,7SEVEN®,11,2,7,45.180121
1,Samsung,36,3,27,30.090983
2,Pigeon,11,1,3,22.590060
3,STRIFF,11,1,1,22.590060
4,Logitech,15,1,9,18.483925
5,Zebronics,16,1,10,17.707583
6,Ambrane,29,1,16,11.728267
7,AGARO,14,0,4,0.000000
8,Amazon,25,0,14,0.000000
9,AmazonBasics,33,0,8,0.000000
